# BespokeFit: Using bespoke force field parameters with OpenFE protocols

This tutorial gives a step-by-step guide on the use of OpenFF-BespokeFit generated force field parameters with OpenFE protocols. Here we will focus on relative
binding free energy (RBFE) calculations, but this strategy can be used with any OpenFE protocol. 

Here we will aussme you have succesfully planned your RBFE campaign using OpenFE by following the [showcase](http://try.openfree.energy/) or [RBFE tutorial](https://docs.openfree.energy/en/stable/tutorials/rbfe_cli_tutorial.html#rbfe-cli-tutorial) and will be using the `TYK2` test system in this example with the planned network provided for you at `inputs/ligand_network.graphml`. 

## BespokeFit submission

First we need to build our BespokeFit fitting protocol and submit the ligands for parameterisation. At this point we assume you already have a running bespokefit server which you can interact with using CLI commands such as: `openff-bespoke executor list` which should indicate that the executor can be reached and is ready to take new jobs. If not you can follow the [bespokefit guide](https://docs.openforcefield.org/projects/bespokefit/en/latest/getting-started/quick-start.html#production-fits) to setup the executor. 

If you have a prefered BespokeFit workflow you can skip this step and load that.

In [1]:
from openff.bespokefit.workflows import BespokeWorkflowFactory
from openff.qcsubmit.common_structures import QCSpec


# create a fast specification built on AIMNET2
aimnet2 = QCSpec(
    method="wb97m-d3",
    basis=None,
    program="aimnet2",
    spec_description="Fast MLQM method using aimnet2."
)

# build the factory using the AIMNET2 specification and set the force field to the newest version of Sage
bespoke_factory = BespokeWorkflowFactory(default_qc_specs=[aimnet2], initial_force_field="openff_unconstrained-2.2.1.offxml")
# update the fitting iterations for large molecules with lots of torsions
bespoke_factory.optimizer.max_iterations = 20
# save the factory to file for later 
bespoke_factory.to_file("aimnet2-bespoke-factory.json")

Now we need to load our planned network and submit each ligand to the BespokeFit server using our chosen bespoke protocol, we should also keep track of the bespokefit task ID which will make it easy to gather the results later.

In [ ]:
from gufe import LigandNetwork, SmallMoleculeComponent, LigandAtomMapping
from openff.bespokefit.executor import BespokeFitClient
from openff.bespokefit.executor.executor import Settings

# load our network file
ligand_network = LigandNetwork.from_graphml(
    open("inputs/ligand_network.graphml", "r").read()
)
# create the client from environment variables
client = BespokeFitClient(settings = Settings())

# for each ligand in the network submit the ligand to bespokefit
name_to_node = {}
for ligand in ligand_network.nodes:
    bespoke_job = bespoke_factory.optimization_schema_from_molecule(
        molecule=ligand.to_openff(), index=ligand.name
    )
    # submit the job and save the task id
    response = client.submit_optimization(input_schema=bespoke_job)
    # we need to round trip the ligand to add the molprop
    ligand_data = ligand.to_dict()
    ligand_data["molprops"]["bespokefit_id"] = response
    new_ligand = SmallMoleculeComponent.from_dict(ligand_data)
    name_to_node[new_ligand.name] = new_ligand

# create a set of new edges using these updated nodes to ensure they are used in the network
edges = []
for edge in ligand_network.edges:
    new_edge = LigandAtomMapping(
        componentA=name_to_node[edge.componentA.name],
        componentB=name_to_node[edge.componentB.name],
        componentA_to_componentB=edge.componentA_to_componentB,
        annotations=edge.annotations
    )
    edges.append(new_edge)

# create a new network from the edges and save to file
new_network = LigandNetwork(edges=edges)
with open("bespoke_ligand_network.graphml", "w") as output:
    output.write(new_network.to_graphml())


Now all we need to do is sit back and wait for the BespokeFit jobs to finish, we can use the code here to load in our ligand network and check the status of the jobs

In [ ]:
import pandas as pd
bespoke_network = LigandNetwork.from_graphml(
    open("bespoke_ligand_network.graphml", "r").read()
)
molecule_data = []
for ligand in bespoke_network.nodes:
    # do we have an easier way to access the molprops?
    ligand_data = ligand.to_dict()
    response = client.get_optimization(optimization_id=ligand_data["molprops"]["bespokefit_id"])
    molecule_data.append(
        {"ligand": ligand.name, "status": response.status, "stage": response.stages[0].type}
    )
print(pd.DataFrame(molecule_data))

## Gathering results

Once the calculations are finished we can use BespokeFit to build a single force field file which contains all of the bespoke parameters for the ligands in this network which we can use with the RBFE protocol.

In [ ]:
# build the command to combine all of the bespoke parameters
command = "openff-bespoke combine --output tyk2_bespoke_ff.offxml "
# extract the ids of the bespokefit jobs for this network
for ligand in bespoke_network.nodes:
    ligand_data = ligand.to_dict()
    command += f"--id {ligand_data['molprops']['bespokefit_id']} "
print(command)

In [ ]:
# run the command to collect all of the results
import os
os.system(command)

## Running the simulations

We are now ready to build a new set of transformations which can be executed locally using the [OpenFE quickrun CLI](https://docs.openfree.energy/en/stable/tutorials/rbfe_cli_tutorial.html#running-the-simulations). We will be following the [python API tutorial](https://docs.openfree.energy/en/stable/tutorials/rbfe_python_tutorial.html#Creating-a-Protocol) to create the transformations, you should check that documentation for more details on the next steps.

In [13]:
# create the OpenFE RBFE protocol using our bespoke force field
from openfe.protocols.openmm_rfe import RelativeHybridTopologyProtocol
from openff.toolkit import ForceField
import openfe

# load the bespokefit force field
force_field = ForceField("tyk2_bespoke_ff.offxml")

# create the default protocol settings
settings = RelativeHybridTopologyProtocol.default_settings()
# add our new force field as a string
# this avoids the need to move the file around when executing the transformations
settings.forcefield_settings.small_molecule_forcefield = force_field.to_string()

# create the protocol
protocol = RelativeHybridTopologyProtocol(settings)

# create the solvent and protein components
solvent = openfe.SolventComponent()
protein = openfe.ProteinComponent.from_pdb_file("inputs/tyk2_protein.pdb")

# follow the tutorial to create the AlchemicalNetwork
transformations = []
for mapping in ligand_network.edges:
    for leg in ['solvent', 'complex']:
        # use the solvent and protein created above
        sysA_dict = {'ligand': mapping.componentA,
                     'solvent': solvent}
        sysB_dict = {'ligand': mapping.componentB,
                     'solvent': solvent}

        if leg == 'complex':
            sysA_dict['protein'] = protein
            sysB_dict['protein'] = protein

        # we don't have to name objects, but it can make things (like filenames) more convenient
        sysA = openfe.ChemicalSystem(sysA_dict, name=f"{mapping.componentA.name}_{leg}")
        sysB = openfe.ChemicalSystem(sysB_dict, name=f"{mapping.componentB.name}_{leg}")

        prefix = "easy_rbfe_"  # prefix is only to exactly reproduce CLI

        transformation = openfe.Transformation(
            stateA=sysA,
            stateB=sysB,
            mapping={'ligand': mapping},
            protocol=protocol,  # use protocol created above
            name=f"{prefix}{sysA.name}_{sysB.name}"
        )
        transformations.append(transformation)

network = openfe.AlchemicalNetwork(transformations)


We can now write out each of the transformations to disk for independent execution: 

In [ ]:
import pathlib
# first we create the directory
transformation_dir = pathlib.Path("transformations")
transformation_dir.mkdir(exist_ok=True)

# then we write out each transformation
for transformation in network.edges:
    transformation.dump(transformation_dir / f"{transformation.name}.json")

# Recap

So to recap the workflow can be reduced to the following steps:
- Plan the RBFE network
- Submit each of the ligands in the network for processing by BespokeFit
- Create a single SMIRNOFF style force field with all of the bespoke parameters for the network using the BespokeFit `combine` CLI
- Store the force field as a string in the OpenFE protocol under the `settings.forcefield_settings.small_molecule_forcefield` field
- Use these settings to create the protcol and create the AlchemicalNetwork following the normal steps

Hopefully its clear that this strategy can be applied to any bespoke parameters you wish to add to the force field not just those from BespokeFit, simply edit your base SMIRNOFF style force field using the [OpenFF-Toolkit](https://docs.openforcefield.org/en/latest/examples/openforcefield/openff-toolkit/forcefield_modification/forcefield_modification.html#modifying-a-smirnoff-force-field) set it in the protocol and simulate! 
